In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import json
from tqdm import tqdm
tqdm.pandas()

In [2]:
#help(pd.read_csv)

In [3]:
data1 = pd.read_csv('lst_announcement_data_1.gzip', compression='gzip')
print('data1 has been read')
data2 = pd.read_csv('lst_announcement_data_2.gzip', compression='gzip')
print('data2 has been read')
data3 = pd.read_csv('lst_announcement_data_3.gzip', compression='gzip')
print('data3 has been read')

data1 has been read
data2 has been read
data3 has been read


In [4]:
%%time
data = pd.concat([data1, data2, data3])

CPU times: user 14.5 s, sys: 2min 15s, total: 2min 29s
Wall time: 7min 55s


In [5]:
del data1
del data2
del data3

In [6]:
data.head()

,id,floornumber,category,bargainterms,description,totalarea,geo,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasgarage,hasshower,bedroomscount,hasdishwasher,repairtype,petsallowed,hasbathhouse,hasfridge,wclocationtype,kitchenarea,roomarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,ptn_dadd
0,191401467,NaN,townhouseSale,"{""price"":5.5E7,""currency"":""rur"",""priceType"":""a...",Все предложения в Барвихе! ОКП Барвиха Клаб. В...,400.00,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""monolith"",""parking"":{},""floor...","{""status"":""individualHousingConstruction"",""are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,5.0,NaN,NaN,NaN,True,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-01
1,212158376,NaN,commercialLandRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...",Сдается от собственника в аренду площадка 560 ...,NaN,"{""countryId"":138,""undergrounds"":[],""calculated...","{""parking"":{},""cranageTypes"":[],""extinguishing...","{""status"":""settlements"",""possibleToChangeStatu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-01
2,227815832,NaN,landSale,"{""price"":240000.0,""currency"":""rur"",""priceType""...",Продам участок в жилом районе с. ХОМУТОВО,NaN,"{""countryId"":138,""undergrounds"":[],""calculated...","{""parking"":{},""cranageTypes"":[],""extinguishing...","{""status"":""individualHousingConstruction"",""are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-01
3,238017518,1.0,flatRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...","Привет, наконец ты попал сюда!\n\nУютная студи...",27.00,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""brick"",""parking"":{},""floorsCo...",{},True,NaN,True,True,NaN,True,True,NaN,NaN,NaN,NaN,euro,False,NaN,True,NaN,NaN,NaN,False,NaN,1.0,NaN,NaN,studio,NaN,2022-08-01
4,241597454,20.0,flatSale,"{""price"":1.6E7,""currency"":""rur"",""priceType"":""a...",Видовой 20 этаж. Корпус Д. Срочная перепродажа...,60.65,"{""countryId"":138,""undergrounds"":[],""calculated...","{""name"":""ЖК «CASABLANKA (Касабланка)»"",""materi...",{},NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,22.0,NaN,False,1.0,1.0,18+20,NaN,rooms,NaN,2022-08-01


In [7]:
def get_nan_cols(df, nan_percent=0.8):
    """
        функция для определения колонок, в которых много NaN
    """
    threshold = len(df.index) * nan_percent
    return [c for c in df.columns if df[c].isna().sum() >= threshold]

colss = get_nan_cols(data, 0.9)
data = data.drop(colss, axis=1)
data.drop(['ptn_dadd'], axis=1, inplace=True) 

In [8]:
data['loc_id'] = data['geo'].progress_apply(lambda x: json.loads(x)['address'][0]['id'])
data['lat'] = data['geo'].progress_apply(lambda x: json.loads(x)['coordinates']['lat'])
data['lng'] = data['geo'].progress_apply(lambda x: json.loads(x)['coordinates']['lng'])
data.drop(['geo'], axis=1, inplace=True)

100%|███████████████████████████████| 2778005/2778005 [05:42<00:00, 8121.00it/s]


In [9]:
data['bargainterms'] = data['bargainterms'].progress_apply(json.loads)
keys_bargainterms = data['bargainterms'].apply(lambda x: x.keys()).explode().unique()
bargainterms_df = pd.DataFrame(data['bargainterms'].values.tolist())

data['building'] = data['building'].progress_apply(json.loads)
keys_building = data['building'].apply(lambda x: x.keys()).explode().unique()
building_df = pd.DataFrame(data['building'].values.tolist())

data['land'] = data['land'].progress_apply(json.loads)
keys_land = data['land'].apply(lambda x: x.keys()).explode().unique()
land_df = pd.DataFrame(data['land'].values.tolist())

land_df.rename(columns={'type': 'land_type'}, inplace=True)

100%|█████████████████████████████| 2778005/2778005 [00:12<00:00, 219031.66it/s]


In [10]:
# удаление строковых колонок
def del_str_cols(df):
    columns = df.columns
    for col in columns:
        if type(df[col].iloc[1]) == str:
            df.drop(col, inplace=True, axis=1)
    return df

print(data.shape)
print(bargainterms_df.shape)
print(building_df.shape)
print(land_df.shape)

data = del_str_cols(data)
bargainterms_df = del_str_cols(bargainterms_df)
building_df = del_str_cols(building_df)
land_df = del_str_cols(land_df)

print(data.shape)
print(bargainterms_df.shape)
print(building_df.shape)
print(land_df.shape)

(2778005, 30)
(2778005, 28)
(2778005, 33)
(2778005, 5)
(2778005, 28)
(2778005, 22)
(2778005, 33)
(2778005, 3)


In [11]:
# почистим колонки с большим кол-вом пропусков и удалим ненужные словарные
data.drop(['bargainterms', 'building', 'land'], axis=1, inplace=True)
colss = get_nan_cols(data, 0.9)
data.drop(colss, axis=1, inplace=True)

colss = get_nan_cols(bargainterms_df, 0.9)
bargainterms_df.drop(colss, axis=1, inplace=True)

colss = get_nan_cols(building_df, 0.9)
building_df.drop(colss, axis=1, inplace=True)

colss = get_nan_cols(land_df, 0.9)
land_df.drop(colss, axis=1, inplace=True)

print(data.shape)
print(bargainterms_df.shape)
print(building_df.shape)
print(land_df.shape)

(2778005, 25)
(2778005, 9)
(2778005, 17)
(2778005, 1)


In [12]:
display(data.head())
display(bargainterms_df.head())
display(building_df.head())
land_df.head()

,id,floornumber,totalarea,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,loc_id,lat,lng
0,191401467,NaN,400.00,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4593,55.718725,37.272026
1,212158376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4579,61.809605,34.296402
2,227815832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4572,52.485696,104.341456
3,238017518,1.0,27.00,True,NaN,True,NaN,True,True,NaN,euro,False,True,NaN,NaN,False,NaN,1.0,NaN,NaN,studio,NaN,4553,45.021564,38.925492
4,241597454,20.0,60.65,NaN,2.0,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,22.0,False,1.0,1.0,18+20,NaN,rooms,NaN,4584,43.485547,39.893921


,price,includedOptions,mortgageAllowed,clientFee,agentFee,prepayMonths,deposit,utilitiesTerms,saleType
0,55000000.0,[],True,NaN,NaN,NaN,NaN,NaN,NaN
1,6000.0,[],NaN,0.0,0.0,1.0,NaN,NaN,NaN
2,240000.0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12990.0,[],NaN,0.0,0.0,1.0,8000.0,"{'includedInPrice': False, 'price': 1.0, 'flow...",NaN
4,16000000.0,[],True,NaN,NaN,NaN,NaN,NaN,free


,materialType,parking,floorsCount,heatingType,cranageTypes,extinguishingSystemTypes,liftTypes,infrastructure,totalArea,openingHours,deadline,name,hasGarbageChute,passengerLiftsCount,buildYear,cargoLiftsCount,ceilingHeight
0,monolith,{},4.0,centralGas,[],[],[],{},400.0,{},{},NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,{},NaN,NaN,[],[],[],{},NaN,{},{},NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,{},NaN,NaN,[],[],[],{},NaN,{},{},NaN,NaN,NaN,NaN,NaN,NaN
3,brick,{},3.0,NaN,[],[],[],{},27.0,{},{},NaN,NaN,NaN,NaN,NaN,NaN
4,monolith,{'type': 'multilevel'},20.0,NaN,[],[],[],{},60.7,{},{},ЖК «CASABLANKA (Касабланка)»,True,3.0,2014.0,NaN,NaN


,area
0,2.0
1,5.0
2,10.0
3,NaN
4,NaN


### Очистка от выбросов


In [214]:
# here is testing
col_name = data['kitchenarea']
print(col_name.min())
print(col_name.max())

KeyError: 'kitchenarea'

In [213]:
# rows to delete
data.drop(data[data['kitchenarea'] == -1], inplace=True, axis=1)

In [215]:
data[data['kitchenarea'] == -1]

KeyError: 'kitchenarea'

In [203]:
### test for defining parameter q
col_name = building_df['kitchenarea']
q = 10
pd.qcut(col_name, q=q)

0                        NaN
1                        NaN
2                        NaN
3                        NaN
4           (2011.0, 2015.0]
                 ...        
2778000                  NaN
2778001     (2004.0, 2011.0]
2778002     (1970.0, 1979.0]
2778003    (999.999, 1961.0]
2778004                  NaN
Name: buildYear, Length: 2778005, dtype: category
Categories (10, interval[float64, right]): [(999.999, 1961.0] < (1961.0, 1970.0] < (1970.0, 1979.0] < (1979.0, 1990.0] ... (2011.0, 2015.0] < (2015.0, 2018.0] < (2018.0, 2021.0] < (2021.0, 2029.0]]

In [ ]:
def my_qcut(col, q):
    tmp = pd.qcut(col, q=q)
    encoder = dict(zip(tmp.unique(), list(range(q + 1))))
    return col.map(encoder)

##### data:

floornumber 9
totalarea 10
hasfurniture BOOL
roomscount 4
haskitchenfurniture	hastv	haswasher	hasbathtub BOOL
bedroomscount 4
repairtype CAT
petsallowed	hasfridge BOOL
wclocationtype CAT
kitchenarea	10
isapartments BOOL
loggiascount 1
balconiescount 3
##allroomsarea DELETE
livingarea 10
flattype CAT
combinedwcscount 2
loc_id - возможно, ранжирование не имеет смысла
lat	10
lng	10

##### bargainterms_df:

price 10
##includedOptions DELETE
mortgageAllowed	BOOL
clientFee 2
agentFee 1
prepayMonths 1
deposit 4	
utilitiesTerms
saleType

##### building_df:

materialType
parking
floorsCount
heatingType
##cranageTypes DELETE
##extinguishingSystemTypes DELETE
##liftTypes DELETE
infrastructure
totalArea
openingHours
deadline	
##name DELETE
hasGarbageChute
passengerLiftsCount
buildYear	
cargoLiftsCount 3
ceilingHeight 10

##### land_df:

area 10